# Playing Valid Chess Moves

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/ShreyaR/guardrails/blob/main/docs/examples/valid_chess_moves.ipynb).

!!! warning
    This example is currently under development (it cannot be used to play a full chess game yet).

In this example, we will use Guardrails to play chess with an LLM and ensure that it makes valid moves.

## Objective

We want to generate a valid chess moves for a given board state.

In [ ]:
import guardrails as gd
from rich import print

In [ ]:
!pip install chess

## Step 1: Create the RAIL Spec

Ordinarily, we would create an RAIL spec in a separate file. For the purposes of this example, we will create the spec in this notebook as a string following the RAIL syntax. For more information on RAIL, see the [RAIL documentation](../rail/output.md).

Here, we request:



In [ ]:
rail_str = """
<rail version="0.1">

<script language='python'>
from dataclasses import dataclass
from guardrails.validators import Validator, EventDetail, register_validator

import re
from typing import Dict, List

import chess

BOARD = chess.Board()

@register_validator(name="is-valid-chess-move", data_type="string")
class IsValidChessMove(Validator):

    board = BOARD

    def validate(self, key, value, schema) -> Dict:
        global BOARD
        try:
            # Push the move onto the board.
            BOARD.push_san(value)
        except Exception as e:
            # If the move is invalid, raise an error.
            raise EventDetail(
                key,
                value,
                schema,
                f"Value {value} is not a valid chess move. {e}",
                None,
            )        

        return schema
</script>


<output>
    <string description="A move in standard algebraic notation." name="move" required="true" format="is-valid-chess-move" on-fail-is-valid-chess-move="reask" />
</output>


<prompt>
Generate a move for the chess board. The board is currently in the following state:
${board_state}
${gr.complete_json_suffix}
</prompt>

</rail>
"""

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

In [ ]:
guard = gd.Guard.from_rail_string(rail_str)

We see the prompt that will be sent to the LLM. The `{board_state}` is substituted with the current state of the board.

In [ ]:
print(guard.base_prompt)

Let's get the reference to the board.

In [ ]:
board = guard.output_schema.move.validators[0].board
board

## Step 3: Wrap the LLM API call with `Guard`

In [ ]:
import openai

raw_llm_response, validated_response = guard(
    openai.Completion.create,
    prompt_params={
        "board_state": str(board.move_stack)
        if board.move_stack
        else "Starting position."
    },
    engine="text-davinci-003",
    max_tokens=2048,
    temperature=0.3,
)

The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary).

We can see that the output is a dictionary with the correct schema and types.

In [ ]:
print(validated_response)

In [ ]:
board

Let's make a move.

In [ ]:
board.push_san("e5")
board

Ask for another move from the model.

In [ ]:
raw_llm_response, validated_response = guard(
    openai.Completion.create,
    prompt_params={
        "board_state": str(board.move_stack)
        if board.move_stack
        else "Starting position."
    },
    engine="text-davinci-003",
    max_tokens=2048,
    temperature=0.3,
)

In [ ]:
board

In [ ]:
board.push_san("Nc6")
board